# Gradient Descent
:label:`sec_gd`

In this section we are going to introduce the basic concepts underlying gradient descent. This is brief by necessity. See e.g., :cite:`Boyd.Vandenberghe.2004` for an in-depth introduction to convex optimization. Although the latter is rarely used directly in deep learning, an understanding of gradient descent is key to understanding stochastic gradient descent algorithms. For instance, the optimization problem might diverge due to an overly large learning rate. This phenomenon can already be seen in gradient descent. Likewise, preconditioning is a common technique in gradient descent and carries over to more advanced algorithms. Let us start with a simple special case.

## Gradient Descent in One Dimension

Gradient descent in one dimension is an excellent example to explain why the gradient descent algorithm may reduce the value of the objective function. Consider some continuously differentiable real-valued function $f: \mathbb{R} \rightarrow \mathbb{R}$. Using a Taylor expansion (:numref:`sec_single_variable_calculus`) we obtain that 

$$f(x + \epsilon) = f(x) + \epsilon f'(x) + \mathcal{O}(\epsilon^2).$$
:eqlabel:`gd-taylor`

That is, in first approximation $f(x+\epsilon)$ is given by the function value $f(x)$ and the first derivative $f'(x)$ at $x$. It is not unreasonable to assume that for small $\epsilon$ moving in the direction of the negative gradient will decrease $f$. To keep things simple we pick a fixed step size $\eta > 0$ and choose $\epsilon = -\eta f'(x)$. Plugging this into the Taylor expansion above we get

$$f(x - \eta f'(x)) = f(x) - \eta f'^2(x) + \mathcal{O}(\eta^2 f'^2(x)).$$

If the derivative $f'(x) \neq 0$ does not vanish we make progress since $\eta f'^2(x)>0$. Moreover, we can always choose $\eta$ small enough for the higher order terms to become irrelevant. Hence we arrive at

$$f(x - \eta f'(x)) \lessapprox f(x).$$

This means that, if we use

$$x \leftarrow x - \eta f'(x)$$

to iterate $x$, the value of function $f(x)$ might decline. Therefore, in gradient descent we first choose an initial value $x$ and a constant $\eta > 0$ and then use them to continuously iterate $x$ until the stop condition is reached, for example, when the magnitude of the gradient $|f'(x)|$ is small enough or the number of iterations has reached a certain value.

For simplicity we choose the objective function $f(x)=x^2$ to illustrate how to implement gradient descent. Although we know that $x=0$ is the solution to minimize $f(x)$, we still use this simple function to observe how $x$ changes. As always, we begin by importing all required modules.

In [1]:
%use @file[../djl.json]
%use lets-plot
import org.jetbrains.letsPlot.intern.Plot
// %load ../utils/djl-imports
// %load ../utils/plot-utils
// %load ../utils/Functions.java

In [2]:
var f = { x: Float -> x * x } // Objective Function
var gradf = { x: Float  -> 2 * x } // Its Derivative

val manager = NDManager.newBaseManager()

Next, we use $x=10$ as the initial value and assume $\eta=0.2$. Using gradient descent to iterate $x$ for 10 times we can see that, eventually, the value of $x$ approaches the optimal solution.

In [3]:
fun gd(eta : Float) : List<Float> {
    var x = 10f
    val results = mutableListOf<Float>()
    results.add(x)
    
    for (i in 0 until 10) {
        x -= eta * gradf(x)
        results.add(x)
    }
    println("epoch 10, x: %f".format(x))
    return results;
}

var res = gd(0.2f)
res

epoch 10, x: 0.060466


[10.0, 6.0, 3.6, 2.1599998, 1.2959999, 0.77759993, 0.46655995, 0.27993596, 0.16796157, 0.10077694, 0.060466163]

The progress of optimizing over $x$ can be plotted as follows.

In [4]:
/* Saved in GradDescUtils.java */
    var n = 0.0f
    for (i in 0 until res.size) {
        if (Math.abs(res[i]) > n) {
            n = Math.abs(res[i])
        }
    }
    val fLineND = manager.arange(-n, n, 0.01f)
    val fLine = fLineND.toFloatArray().toList()
    // plotGD(fLine, Functions.callFunc(fLine, f), res, f, 500, 400);

    fun plotGD(fLine: List<Float>, res: List<Float>, func: (Float)-> Float, width : Int, height: Int) : Plot {
        var data = mapOf( "x" to fLine,
                "fx" to fLine.map(func)
                 )
var data1 = mapOf("f" to res.map(func),
                 "x1" to res
                )
var plot = letsPlot()
plot += geomLine(data=data) { x = "x" ; y = "fx" }
plot += geomLine(data=data1, color = "red") { x="x1"; y = "f"}
plot += geomPoint(data=data1, size=3.0) { x="x1"; y = "f"}
return (plot + ggsize(width, height))
    }
//showTrace(res)

plotGD(fLine, res, f, 500,400)

### Learning Rate
:label:`section_gd-learningrate`

The learning rate $\eta$ can be set by the algorithm designer. If we use a learning rate that is too small, it will cause $x$ to update very slowly, requiring more iterations to get a better solution. To show what happens in such a case, consider the progress in the same optimization problem for $\eta = 0.05$. As we can see, even after 10 steps we are still very far from the optimal solution.

In [5]:
res = gd(0.05f)
println(res)
    var n = 0.0f
    for (i in 0 until res.size) {
        if (Math.abs(res[i]) > n) {
            n = Math.abs(res[i])
        }
    }
    val fLineND = manager.arange(-n, n, 0.01f)
    val fLine = fLineND.toFloatArray().toList()
    // plotGD(fLine, Functions.callFunc(fLine, f), res, f, 500, 400);

//showTrace(res)

plotGD(fLine, res, f, 500,400)

epoch 10, x: 3.486785
[10.0, 9.0, 8.1, 7.2900004, 6.5610003, 5.9049006, 5.3144107, 4.7829695, 4.3046727, 3.8742054, 3.486785]


Conversely, if we use an excessively high learning rate, $\left|\eta f'(x)\right|$ might be too large for the first-order Taylor expansion formula. That is, the term $\mathcal{O}(\eta^2 f'^2(x))$ in :eqref:`gd-taylor` might become significant. In this case, we cannot guarantee that the iteration of $x$ will be able to lower the value of $f(x)$. For example, when we set the learning rate to $\eta=1.1$, $x$ overshoots the optimal solution $x=0$ and gradually diverges.

In [6]:
res = gd(1.1f)
println(res)
    var n = 0.0f
    for (i in 0 until res.size) {
        if (Math.abs(res[i]) > n) {
            n = Math.abs(res[i])
        }
    }
    val fLineND = manager.arange(-n, n, 0.01f)
    val fLine = fLineND.toFloatArray().toList()
    // plotGD(fLine, Functions.callFunc(fLine, f), res, f, 500, 400);

plotGD(fLine, res, f, 500,400)

epoch 10, x: 61.917389
[10.0, -12.0, 14.400002, -17.280003, 20.736004, -24.883205, 29.859848, -35.831818, 42.998184, -51.597824, 61.91739]


### Local Minima

To illustrate what happens for nonconvex functions consider the case of $f(x) = x \cdot \cos c x$. This function has infinitely many local minima. Depending on our choice of learning rate and depending on how well conditioned the problem is, we may end up with one of many solutions. The example below illustrates how an (unrealistically) high learning rate will lead to a poor local minimum.

In [7]:
var c = 0.15f * Math.PI

f = { x: Float -> x * Math.cos(c * x).toFloat() }

gradf = { x: Float -> (Math.cos(c * x) - c * x * Math.sin(c * x)).toFloat() }

res = gd(2.0f)
println(res)

var n = res.map { Math.abs(it) }.max()
val fLine = manager.arange(-n, n, 0.01f).toFloatArray().toList()
plotGD(fLine, res, f, 500,400)

epoch 10, x: -1.528166
[10.0, 0.57522106, -1.2065896, -2.2795973, -1.3434169, -2.2067688, -1.4257898, -2.1546335, -1.4838762, -2.1143532, -1.5281663]


## Multivariate Gradient Descent

Now that we have a better intuition of the univariate case, let us consider the situation where $\mathbf{x} \in \mathbb{R}^d$. That is, the objective function $f: \mathbb{R}^d \to \mathbb{R}$ maps vectors into scalars. Correspondingly its gradient is multivariate, too. It is a vector consisting of $d$ partial derivatives:

$$\nabla f(\mathbf{x}) = \bigg[\frac{\partial f(\mathbf{x})}{\partial x_1}, \frac{\partial f(\mathbf{x})}{\partial x_2}, \ldots, \frac{\partial f(\mathbf{x})}{\partial x_d}\bigg]^\top.$$

Each partial derivative element $\partial f(\mathbf{x})/\partial x_i$ in the gradient indicates the rate of change of $f$ at $\mathbf{x}$ with respect to the input $x_i$. As before in the univariate case we can use the corresponding Taylor approximation for multivariate functions to get some idea of what we should do. In particular, we have that 

$$f(\mathbf{x} + \mathbf{\epsilon}) = f(\mathbf{x}) + \mathbf{\epsilon}^\top \nabla f(\mathbf{x}) + \mathcal{O}(\|\mathbf{\epsilon}\|^2).$$
:eqlabel:`gd-multi-taylor`

In other words, up to second order terms in $\mathbf{\epsilon}$ the direction of steepest descent is given by the negative gradient $-\nabla f(\mathbf{x})$. Choosing a suitable learning rate $\eta > 0$ yields the prototypical gradient descent algorithm:

$\mathbf{x} \leftarrow \mathbf{x} - \eta \nabla f(\mathbf{x}).$

To see how the algorithm behaves in practice let us construct an objective function $f(\mathbf{x})=x_1^2+2x_2^2$ with a two-dimensional vector $\mathbf{x} = [x_1, x_2]^\top$ as input and a scalar as output. The gradient is given by $\nabla f(\mathbf{x}) = [2x_1, 4x_2]^\top$. We will observe the trajectory of $\mathbf{x}$ by gradient descent from the initial position $[-5, -2]$. We need two more helper functions. The first uses an update function and applies it $20$ times to the initial value. The second helper visualizes the trajectory of $\mathbf{x}$.

We also create a `Weights` class to make it easier to store the weight parameters and return them in functions.

In [8]:
/* Saved in GradDescUtils.java */
class Weights(val x1: Float, val x2: Float) {
}

/* Saved in GradDescUtils.java */
/* Optimize a 2D objective function with a customized trainer. */
fun train2d(trainer: (List<Float>)->List<Float>, steps: Int) : List<Weights>{
    // s1 and s2 are internal state variables and will
    // be used later in the chapter
    var x1 = -5f
    var x2 = -2f
    var s1 = 0f
    var s2 = 0f
    val results = mutableListOf<Weights>()
    results.add(Weights(x1, x2))
    for (i in 1 until steps + 1) {
        val step = trainer(listOf<Float>(x1, x2, s1, s2))
        x1 = step[0];
        x2 = step[1];
        s1 = step[2];
        s2 = step[3];
        results.add(Weights(x1, x2));
        println("epoch %d, x1 %f, x2 %f".format(i, x1, x2))
    }
    return results
}

/* Saved in GradDescUtils.java */
fun meshgridPoints(x:List<Double>, y:List<Double>): List<kotlin.Pair<Double, Double>> {
  val xSer = y.flatMap { x }
  var yInd = -1
  val ySer = y.flatMap { 
      yInd++
      List(x.size) {y[yInd]} 
  }
  return xSer.zip(ySer)
}

var gridPoints = meshgridPoints(
    generateSequence(-5.5, {it + 15.0/50} ).takeWhile { it <= 0.5 }.toList(),
    generateSequence(-3.0, {it + 15.0/40} ).takeWhile { it <= 0.5 }.toList()
)

/* Show the trace of 2D variables during optimization. */        
fun showTrace2d(f: (Float, Float)-> Float, results: List<Weights>) {
    // TODO: add when tablesaw adds support for contour and meshgrids
}

Next, we observe the trajectory of the optimization variable $\mathbf{x}$ for learning rate $\eta = 0.1$. We can see that after 20 steps the value of $\mathbf{x}$ approaches its minimum at $[0, 0]$. Progress is fairly well-behaved albeit rather slow.

In [17]:
var eta = 0.1f;

val ff = fun(x1: Float, x2: Float) : Float { return x1 * x1 + 2 * x2 * x2 }  // Objective

val gradff = fun(x1: Float, x2: Float) : List<Float> { return listOf(2 * x1, 4 * x2) } // Gradient

val gd = { state : List<Float> -> 
    val x1 = state[0]
    val x2 = state[1]
    
    val g = gradff(x1, x2) // Compute Gradient
    val g1 = g[0]
    val g2 = g[1]
    
    listOf<Float>(x1 - eta * g1, x2 - eta * g2, 0f, 0f) // Update Variables
}

//showTrace2d(ff, train2d(gd, 20))
val tr = train2d(gd,20)
//println(tr)
val dd = mapOf( "x" to tr.map { it.x1 }, "y" to tr.map { it.x2 }, "z" to tr.map { ff(it.x1, it.x2) })
val dd3 = mapOf("x" to gridPoints.map { it.first }, "y" to gridPoints.map { it.second }, "z" to gridPoints.map { ff(it.first.toFloat(), it.second.toFloat())})

var plot = letsPlot()
plot += geomLine(data=dd) { x = "x" ; y = "y" }
plot += geomPoint(data=dd, size=3) { x = "x" ; y = "y" }
plot += geomContour(data=dd3, color="red", binWidth = 7.0) {x="x"; y="y"; z="z"}
plot + ggsize(500, 400)

epoch 1, x1 -4.000000, x2 -1.200000
epoch 2, x1 -3.200000, x2 -0.720000
epoch 3, x1 -2.560000, x2 -0.432000
epoch 4, x1 -2.048000, x2 -0.259200
epoch 5, x1 -1.638400, x2 -0.155520
epoch 6, x1 -1.310720, x2 -0.093312
epoch 7, x1 -1.048576, x2 -0.055987
epoch 8, x1 -0.838861, x2 -0.033592
epoch 9, x1 -0.671089, x2 -0.020155
epoch 10, x1 -0.536871, x2 -0.012093
epoch 11, x1 -0.429497, x2 -0.007256
epoch 12, x1 -0.343597, x2 -0.004354
epoch 13, x1 -0.274878, x2 -0.002612
epoch 14, x1 -0.219902, x2 -0.001567
epoch 15, x1 -0.175922, x2 -0.000940
epoch 16, x1 -0.140737, x2 -0.000564
epoch 17, x1 -0.112590, x2 -0.000339
epoch 18, x1 -0.090072, x2 -0.000203
epoch 19, x1 -0.072058, x2 -0.000122
epoch 20, x1 -0.057646, x2 -0.000073


![Contour Gradient Descent.](https://d2l-java-resources.s3.amazonaws.com/img/contour_gd.svg)

## Adaptive Methods

As we could see in :numref:`section_gd-learningrate`, getting the learning rate $\eta$ "just right" is tricky. If we pick it too small, we make no progress. If we pick it too large, the solution oscillates and in the worst case it might even diverge. What if we could determine $\eta$ automatically or get rid of having to select a step size at all? Second order methods that look not only at the value and gradient of the objective but also at its *curvature* can help in this case. While these methods cannot be applied to deep learning directly due to the computational cost, they provide useful intuition into how to design advanced optimization algorithms that mimic many of the desirable properties of the algorithms outlined below.


### Newton's Method

Reviewing the Taylor expansion of $f$ there is no need to stop after the first term. In fact, we can write it as 

$$f(\mathbf{x} + \mathbf{\epsilon}) = f(\mathbf{x}) + \mathbf{\epsilon}^\top \nabla f(\mathbf{x}) + \frac{1}{2} \mathbf{\epsilon}^\top \nabla \nabla^\top f(\mathbf{x}) \mathbf{\epsilon} + \mathcal{O}(\|\mathbf{\epsilon}\|^3).$$
:eqlabel:`gd-hot-taylor`

To avoid cumbersome notation we define $H_f := \nabla \nabla^\top f(\mathbf{x})$ to be the *Hessian* of $f$. This is a $d \times d$ matrix. For small $d$ and simple problems $H_f$ is easy to compute. For deep networks, on the other hand, $H_f$ may be prohibitively large, due to the cost of storing $\mathcal{O}(d^2)$ entries. Furthermore it may be too expensive to compute via backprop as we would need to apply backprop to the backpropagation call graph. For now let us ignore such considerations and look at what algorithm we'd get. 

After all, the minimum of $f$ satisfies $\nabla f(\mathbf{x}) = 0$. Taking derivatives of :eqref:`gd-hot-taylor` with regard to $\mathbf{\epsilon}$ and ignoring higher order terms we arrive at 

$$\nabla f(\mathbf{x}) + H_f \mathbf{\epsilon} = 0 \text{ and hence } 
\mathbf{\epsilon} = -H_f^{-1} \nabla f(\mathbf{x}).$$

That is, we need to invert the Hessian $H_f$ as part of the optimization problem.

For $f(x) = \frac{1}{2} x^2$ we have $\nabla f(x) = x$ and $H_f = 1$. Hence for any $x$ we obtain $\epsilon = -x$. In other words, a single step is sufficient to converge perfectly without the need for any adjustment! Alas, we got a bit lucky here since the Taylor expansion was exact. Let us see what happens in other problems.

In [14]:
var c = 0.5

f = { x: Float -> Math.cosh(c * x).toFloat() } // Objective

gradf = { x: Float -> (c * Math.sinh(c * x)).toFloat() }  // Derivative

var hessf = { x: Float -> (c * c * Math.cosh(c * x)).toFloat() }  // Hessian

// Hide learning rate for now
fun newton(eta: Float): List<Float> {
    var x = 10f
    val results = mutableListOf<Float>()
    results.add(x)
    
    for (i in 0 until 10) {
        x -= eta * gradf(x) / hessf(x);
        results.add(x)
    }
    println("epoch 10, x: %f".format(x))
    return results;
}

res = newton(1.0f)
println(res)

var n = res.map { Math.abs(it) }.max()
val fLine = manager.arange(-n, n, 0.01f).toFloatArray().toList()
plotGD(fLine, res, f, 500,400)

epoch 10, x: 0.000000
[10.0, 8.000181, 6.0015225, 4.0113983, 2.0825424, 0.5257633, 0.011785626, 1.3690442E-7, 0.0, 0.0, 0.0]


Now let us see what happens when we have a *nonconvex* function, such as $f(x) = x \cos(c x)$. After all, note that in Newton's method we end up dividing by the Hessian. This means that if the second derivative is *negative* we would walk into the direction of *increasing* $f$. That is a fatal flaw of the algorithm. Let us see what happens in practice.

In [15]:
c = 0.15f * Math.PI

f = { x: Float -> (x * Math.cos(c * x)).toFloat() }

gradf = { x: Float -> (Math.cos(c * x) - c * x * Math.sin(c * x)).toFloat() }

hessf = { x: Float -> (-2 * c * Math.sin(c * x) - x * c * c * Math.cos(c * x)).toFloat() }
                                    
res = newton(1.0f)
println(res)

var n = res.map { Math.abs(it) }.max()
val fLine = manager.arange(-n, n, 0.01f).toFloatArray().toList()
plotGD(fLine, res, f, 500,400)

epoch 10, x: 26.834131
[10.0, 4.9999976, 24.99519, 27.782484, 26.800192, 26.834177, 26.834131, 26.834131, 26.834131, 26.834131, 26.834131]


This went spectacularly wrong. How can we fix it? One way would be to "fix" the Hessian by taking its absolute value instead. Another strategy is to bring back the learning rate. This seems to defeat the purpose, but not quite. Having second order information allows us to be cautious whenever the curvature is large and to take longer steps whenever the objective is flat. Let us see how this works with a slightly smaller learning rate, say $\eta = 0.5$. As we can see, we have quite an efficient algorithm.

In [16]:
res = newton(0.5f)
println(res)

var n = res.map { Math.abs(it) }.max()
val fLine = manager.arange(-n, n, 0.01f).toFloatArray().toList()
plotGD(fLine, res, f, 500,400)

epoch 10, x: 7.269860
[10.0, 7.499999, 7.387164, 7.329002, 7.299392, 7.2844415, 7.276928, 7.2731614, 7.2712755, 7.270332, 7.26986]


### Convergence Analysis

We only analyze the convergence rate for convex and three times differentiable $f$, where at its minimum $x^*$ the second derivative is nonzero, i.e., where $f''(x^*) > 0$. The multivariate proof is a straightforward extension of the argument below and omitted since it doesn't help us much in terms of intuition. 

Denote by $x_k$ the value of $x$ at the $k$-th iteration and let $e_k := x_k - x^*$ be the distance from optimality. By Taylor series expansion we have that the condition $f'(x^*) = 0$ can be written as

$$0 = f'(x_k - e_k) = f'(x_k) - e_k f''(x_k) + \frac{1}{2} e_k^2 f'''(\xi_k).$$

This holds for some $\xi_k \in [x_k - e_k, x_k]$. Recall that we have the update $x_{k+1} = x_k - f'(x_k) / f''(x_k)$. Dividing the above expansion by $f''(x_k)$ yields

$$e_k - f'(x_k) / f''(x_k) = \frac{1}{2} e_k^2 f'''(\xi_k) / f''(x_k).$$ 

Plugging in the update equations leads to the following bound $e_{k+1} \leq e_k^2 f'''(\xi_k) / f'(x_k)$. Consequently, whenever we are in a region of bounded $f'''(\xi_k) / f''(x_k) \leq c$, we have a quadratically decreasing error $e_{k+1} \leq c e_k^2$. 

As an aside, optimization researchers call this *linear* convergence, whereas a condition such as $e_{k+1} \leq \alpha e_k$ would be called a *constant* rate of convergence. 
Note that this analysis comes with a number of caveats: We do not really have much of a guarantee when we will reach the region of rapid convergence. Instead, we only know that once we reach it, convergence will be very quick. Second, this requires that $f$ is well-behaved up to higher order derivatives. It comes down to ensuring that $f$ does not have any "surprising" properties in terms of how it might change its values. 

### Preconditioning

Quite unsurprisingly computing and storing the full Hessian is very expensive. It is thus desirable to find alternatives. One way to improve matters is by avoiding to compute the Hessian in its entirety but only compute the *diagonal* entries. While this is not quite as good as the full Newton method, it is still much better than not using it. Moreover, estimates for the main diagonal elements are what drives some of the innovation in stochastic gradient descent optimization algorithms. This leads to update algorithms of the form

$$\mathbf{x} \leftarrow \mathbf{x} - \eta \mathrm{diag}(H_f)^{-1} \nabla \mathbf{x}.$$

To see why this might be a good idea consider a situation where one variable denotes height in millimeters and the other one denotes height in kilometers. Assuming that for both the natural scale is in meters we have a terrible mismatch in parameterizations. Using preconditioning removes this. Effectively preconditioning with gradient descent amounts to selecting a different learning rate for each coordinate. 

### Gradient Descent with Line Search

One of the key problems in gradient descent was that we might overshoot the goal or make insufficient progress. A simple fix for the problem is to use line search in conjunction with gradient descent. That is, we use the direction given by $\nabla f(\mathbf{x})$ and then perform binary search as to which step length $\eta$ minimizes $f(\mathbf{x} - \eta \nabla f(\mathbf{x}))$. 

This algorithm converges rapidly (for an analysis and proof see e.g., :cite:`Boyd.Vandenberghe.2004`). However, for the purpose of deep learning this is not quite so feasible, since each step of the line search would require us to evaluate the objective function on the entire dataset. This is way too costly to accomplish. 

## Summary

* Learning rates matter. Too large and we diverge, too small and we do not make progress.
* Gradient descent can get stuck in local minima.
* In high dimensions adjusting learning the learning rate is complicated.
* Preconditioning can help with scale adjustment.
* Newton's method is a lot faster *once* it has started working properly in convex problems.
* Beware of using Newton's method without any adjustments for nonconvex problems. 

## Exercises

1. Experiment with different learning rates and objective functions for gradient descent.
1. Implement line search to minimize a convex function in the interval $[a, b]$.
    * Do you need derivatives for binary search, i.e., to decide whether to pick $[a, (a+b)/2]$ or $[(a+b)/2, b]$. 
    * How rapid is the rate of convergence for the algorithm?
    * Implement the algorithm and apply it to minimizing $\log (\exp(x) + \exp(-2*x -3))$.
1. Design an objective function defined on $\mathbb{R}^2$ where gradient descent is exceedingly slow. Hint - scale different coordinates differently.
1. Implement the lightweight version of Newton's method using preconditioning:
    * Use diagonal Hessian as preconditioner.
    * Use the absolute values of that rather than the actual (possibly signed) values. 
    * Apply this to the problem above.
1. Apply the algorithm above to a number of objective functions (convex or not). What happens if you rotate coordinates by $45$ degrees?